In [178]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

customer = pd.read_csv("Customer.csv")
countryPopulation = pd.read_csv("CountryPopulation.csv")
countryGDP = pd.read_csv("CountryGDP.csv")


# Devoir 2 - 8IAR403
Étudiant : *Mael Garnier*


Plan :
    1. Réalisation de l'entrainement du modèle



## 1. Préparation

### 1.1 Pipeline de transformation

Nous avions déjà réalisé le pipeline de transformation lors du devoir 1. Nous allons donc le réutiliser ici.

Tout le code suivant est une copie condensée du code du devoir 1.

In [179]:
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


def replace_missing_values_function(X, cols_to_numeric, cols_to_exclude):
    for col in cols_to_numeric:
        if col in X.columns and X[col] is not None:
            X[col] = pd.to_numeric(X[col], errors='coerce')

    imputer = SimpleImputer(strategy="median")
    x_num = X.drop(columns=cols_to_exclude, axis=1)
    imputer.fit(x_num)
    x_num_tr = imputer.transform(x_num)
    x_tr = pd.DataFrame(x_num_tr, columns=x_num.columns,
                          index=X.index)
    x_tr[cols_to_exclude] = X[cols_to_exclude]
    return x_tr
replace_missing_values_customer = FunctionTransformer(
    replace_missing_values_function,
    validate=False,
    kw_args={
        "cols_to_numeric": ["first_item_prize", "revenue"],
        "cols_to_exclude": ["gender", "ReBuy", "country"]
    }
)
def replace_outliers_with_interpolation_function(X, cols_to_exclude):
    x_num = X.drop(columns=cols_to_exclude, axis=1)
    Q1 = x_num.quantile(0.25)
    Q3 = x_num.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    for column in x_num.columns:
        x_num[column] = np.where(
            (x_num[column] < lower_bound[column]) | (x_num[column] > upper_bound[column]),
            np.nan,
            x_num[column]
        )
        x_num[column] = x_num[column].interpolate(method='slinear', limit_direction='both')
    x_num[cols_to_exclude] = X[cols_to_exclude]
    return x_num
replace_outliers_customer = FunctionTransformer(
    replace_outliers_with_interpolation_function,
    validate=False,
    kw_args={
        "cols_to_exclude": ["gender", "ReBuy", "country", "age"]
    }
)
def data_enrichment_function(X, addPIB):
    new_countryPopulation = replace_missing_values_function(countryPopulation, cols_to_numeric=[], cols_to_exclude=["Country"])
    new_countryPopulation.rename(columns={'Country': 'country'}, inplace=True)
    X['country'] = X['country'].str.lower()
    new_countryPopulation['country'] = new_countryPopulation['country'].str.lower()
    merged_df = pd.merge(X, new_countryPopulation, on='country', how='left')
    if addPIB :
        new_countryGDP = replace_missing_values_function(countryGDP, cols_to_numeric=[], cols_to_exclude=["Country"])
        new_countryGDP.rename(columns={'Country': 'country'}, inplace=True)
        new_countryGDP['country'] = new_countryGDP['country'].str.lower()
        merged_df = pd.merge(merged_df, new_countryGDP, on='country', how='left')
    return merged_df
data_enrichment = FunctionTransformer(
    data_enrichment_function,
    validate=False,
    kw_args={
        "addPIB": True
    },
)
data_enrichment_nopib = FunctionTransformer(
    data_enrichment_function,
    validate=False,
    kw_args={
        "addPIB": False
    },
)
data_cleaning = Pipeline([
    ("replace_missing", replace_missing_values_customer),
    ("replace_outliers", replace_outliers_customer),
])
def remove_non_numeric_columns(arr):
        df = pd.DataFrame(arr)
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                df.drop(columns=[col], inplace=True)
        return df.to_numpy()
def prepare_customer(X):
    full_pipeline = ColumnTransformer(
        transformers=[
            ("cleaning", data_cleaning, X.columns),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include=np.number))
        ],
        remainder="drop"
    )
    res = full_pipeline.fit_transform(X)
    return remove_non_numeric_columns(res)
def prepare_customer_pop(X):
    full_pipeline = ColumnTransformer(
        transformers=[
            ("cleaning", data_cleaning, X.columns),
            ("enrichment", data_enrichment_nopib, X.columns),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include=np.number))
        ],
        remainder="drop"
    )
    res = full_pipeline.fit_transform(X)
    return remove_non_numeric_columns(res)
def prepare_customer_pop_gdp(X):
    full_pipeline = ColumnTransformer(
        transformers=[
            ("cleaning", data_cleaning, X.columns),
            ("enrichment", data_enrichment, X.columns),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include=np.number))
        ],
        remainder="drop"
    )
    res = full_pipeline.fit_transform(X)
    return remove_non_numeric_columns(res)

### 1.2. Séparation des données

Nous allons séparer les données en données d'entrainement et données de test.

In [180]:
train_set, test_set = train_test_split(customer, test_size=0.2,
                                       random_state=5)

## 2. Entrainement d'un classeur binaire

### 2.1 Préparation des données d'entrainement

Nous allons préparer les données d'entrainement. Commencons par isoler la classe binaire.

In [181]:
# Remplacement des valeurs manquantes sur le jeu de données pour préparer la classe
train_set_corrected = replace_missing_values_function(train_set, ["first_item_prize", "revenue"], ["gender", "ReBuy", "country"])
# Calcul de la moyenne des revenus depuis le jeu de données corrigé
revenue_mean = train_set_corrected["revenue"].mean()
# Création de la classe binaire, 1 si le revenu est supérieur à la moyenne, 0 sinon
customer_labels = (train_set_corrected["revenue"] > revenue_mean).astype(int)

customer_labels.head()

7751    1
4154    0
3881    0
9238    1
5210    0
Name: revenue, dtype: int64

On peut voir que la classe a bien été préparée avec des valeurs binaires : 0 si le revenu est inférieur à la moyenne, 1 sinon.

Nous pouvons maintenant préparer les données d'entrainement avec le pipeline, en prenant soin de retirer la classe binaire.

In [182]:

# Préparation des données d'entrainement
customer_train = resample(train_set, replace=False, n_samples=8000, random_state=42)
customer_train = customer_train.drop("revenue", axis=1)

# Traitement des données avec le pipeline

# Uniquement customer
customer_prepare = prepare_customer(customer_train)
# customer + population
customer_pop_prepare = prepare_customer_pop(customer_train)
# customer + population + gdp
customer_pop_gdp_prepare = prepare_customer_pop_gdp(customer_train)


### 2.2 Entrainement du modèle

Nous allons entrainer un modèle de classification binaire.

In [183]:
# Entrainement du modèle
tree_class = DecisionTreeClassifier()
tree_class.fit(customer_prepare, customer_labels)

DecisionTreeClassifier()

In [184]:
# Fonction pour calculer la précision, le rappel et la F1-mesure dans le cadre de la validation croisée
def cross_val_classification_metrics(classifier, X, y, cv=3):
    # Définir les scores personnalisés
    precision_scorer = make_scorer(precision_score)
    recall_scorer = make_scorer(recall_score)
    f1_scorer = make_scorer(f1_score)

    # Validation croisée avec précision
    precision_scores = cross_val_score(classifier, X, y, scoring=precision_scorer, cv=cv)
    print(f"Précision (k={cv}): {precision_scores.mean()} ± {precision_scores.std()}")

    # Validation croisée avec rappel
    recall_scores = cross_val_score(classifier, X, y, scoring=recall_scorer, cv=cv)
    print(f"Rappel (k={cv}): {recall_scores.mean()} ± {recall_scores.std()}")

    # Validation croisée avec F1-mesure
    f1_scores = cross_val_score(classifier, X, y, scoring=f1_scorer, cv=cv)
    print(f"F1-mesure (k={cv}): {f1_scores.mean()} ± {f1_scores.std()}")

# Evaluation du modèle pour chaque ensemble de données avec k=3 et k=10
print("Évaluation avec k=3")

# Evaluation pour les différentes configurations de données
print("\nCustomer (k=3):")
cross_val_classification_metrics(tree_class, customer_prepare, customer_labels, cv=3)

print("\nCustomer + Population (k=3):")
cross_val_classification_metrics(tree_class, customer_pop_prepare, customer_labels, cv=3)

print("\nCustomer + Population + GDP (k=3):")
cross_val_classification_metrics(tree_class, customer_pop_gdp_prepare, customer_labels, cv=3)

print("\nÉvaluation avec k=10")

print("\nCustomer (k=10):")
cross_val_classification_metrics(tree_class, customer_prepare, customer_labels, cv=10)

print("\nCustomer + Population (k=10):")
cross_val_classification_metrics(tree_class, customer_pop_prepare, customer_labels, cv=10)

print("\nCustomer + Population + GDP (k=10):")
cross_val_classification_metrics(tree_class, customer_pop_gdp_prepare, customer_labels, cv=10)

Évaluation avec k=3

Customer (k=3):
Précision (k=3): 0.40404132319382446 ± 0.008970028393106377
Rappel (k=3): 0.3760793785938155 ± 0.007840562807381005
F1-mesure (k=3): 0.3949355889001091 ± 0.006029985267291585

Customer + Population (k=3):
Précision (k=3): 0.39031195448346234 ± 0.007466457810313703
Rappel (k=3): 0.407500740356852 ± 0.020353561376429803
F1-mesure (k=3): 0.3980828106641485 ± 0.017290969259432493

Customer + Population + GDP (k=3):
Précision (k=3): 0.39256290680643185 ± 0.007693727846509375
Rappel (k=3): 0.40044606500333163 ± 0.016869970626995464
F1-mesure (k=3): 0.3974491656123727 ± 0.010386290981080345

Évaluation avec k=10

Customer (k=10):
Précision (k=10): 0.3904566444710018 ± 0.01802639635657764
Rappel (k=10): 0.36933485860334736 ± 0.024628897748793434
F1-mesure (k=10): 0.3792023470519832 ± 0.014782107352125002

Customer + Population (k=10):
Précision (k=10): 0.3877564339575469 ± 0.015731196457890576
Rappel (k=10): 0.39436268447522466 ± 0.019344434667352865
F1-mes

In [185]:
# Définir l'intervalle des hyperparamètres à tester
param_grid = {
    'max_depth': [3, 5, 7, 10, 15, None],  # Profondeur de l'arbre
    'min_samples_split': [2, 5, 10, 20]   # Nombre minimal d'observations pour une division
}

# Création du modèle d'arbre de décision
tree_class = DecisionTreeClassifier(random_state=42)

# Initialisation de GridSearchCV avec la validation croisée de k blocs
grid_search = GridSearchCV(estimator=tree_class, param_grid=param_grid, cv=3, scoring='accuracy')

def grid_search_function(numpy, labels):
    grid_search.fit(numpy, labels)
    # Meilleurs paramètres trouvés
    print("Meilleurs paramètres trouvés : ", grid_search.best_params_)
    # Meilleurs scores
    print("Meilleur score (accuracy) : ", grid_search.best_score_)

print("Customer")
grid_search_function(customer_prepare, customer_labels)

print("\nCustomer + Population")
grid_search_function(customer_pop_prepare, customer_labels)

print("\nCustomer + Population + GDP")
grid_search_function(customer_pop_gdp_prepare, customer_labels)

Customer
Meilleurs paramètres trouvés :  {'max_depth': 3, 'min_samples_split': 2}
Meilleur score (accuracy) :  0.6089997846293219

Customer + Population
Meilleurs paramètres trouvés :  {'max_depth': 3, 'min_samples_split': 2}
Meilleur score (accuracy) :  0.6095000503031645

Customer + Population + GDP
Meilleurs paramètres trouvés :  {'max_depth': 3, 'min_samples_split': 2}
Meilleur score (accuracy) :  0.606875378387154


In [193]:
# Remplacement des valeurs manquantes sur le jeu de données pour préparer la classe
test_set_corrected = replace_missing_values_function(test_set, ["first_item_prize", "revenue"], ["gender", "ReBuy", "country"])
# Calcul de la moyenne des revenus depuis le jeu de données corrigé
revenue_mean = test_set_corrected["revenue"].mean()
# Création de la classe binaire, 1 si le revenu est supérieur à la moyenne, 0 sinon
customer_labels = (test_set_corrected["revenue"] > revenue_mean).astype(int)

# Préparation des données d'entrainement
customer_test = resample(test_set, replace=False, n_samples=2000, random_state=42)
customer_test = customer_test.drop("revenue", axis=1)

# Traitement des données avec le pipeline

# Uniquement customer
full_pipeline = ColumnTransformer(
    transformers=[
        ("cleaning", data_cleaning, customer_test.columns),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include=np.number))
    ],
    remainder="drop"
)
customer_prepare = full_pipeline.transform(customer_test)
customer_prepare = remove_non_numeric_columns(customer_prepare)
pd.DataFrame(customer_prepare).head()


NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.